In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pypng


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os

# TODO: Fill in the Google Drive path where you uploaded the assignment
# Example: If you create a 2025Fall folder and put all the files under A3 folder, then '2025Fall/A3'

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/Computer Vision/Projects/MLP'

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

FileNotFoundError: [Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/Computer Vision/Projects/MLP'

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/Computer\ Vision/Projects/MLP
!git clone https://github.com/rafaelcp/smbdataset.git
!mkdir -p extracted
!7z x /content/drive/MyDrive/Colab\ Notebooks/Computer\ Vision/Projects/MLP/smbdataset/data-smb.7z -oextracted -y

/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP
fatal: destination path 'smbdataset' already exists and is not an empty directory.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/Colab No . Vision/Projects/MLP/smbdataset/                                                                           1 file, 932651968 bytes (890 MiB)

Extracting archive: /content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/smbdataset/data-smb.7z
--
Path = /content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/smbdataset/data-smb.7z
Type = 7z
Physical Size = 932651968
Headers Size = 6735576
Method = LZM

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab\ Notebooks/Computer\ Vision/Projects/MLP/smbdataset');
!cp /content/drive/MyDrive/Colab\ Notebooks/Computer\ Vision/Projects/gamengen-wm/config_sd.py .

In [ ]:
from pathlib import Path
import re, os, random
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import png
from diffusers import AutoencoderKL
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from config_sd import HEIGHT, WIDTH

device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
data_root = Path("/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/extracted")
max_files = 2000
ctx = 4
frameskip = 4
torch.manual_seed(0)



In [ ]:
name_re = re.compile(r"_f(?P<frame>\d+)_a(?P<action>\d+)_.*\.(?P<outcome>win|fail)\.png")

def parse_outcome(path: Path):
    m = name_re.search(path.name)
    return 1.0 if m and m.group("outcome") == "win" else 0.0

def read_ram(path: Path):
    try:
        b = path.read_bytes()
        start = b.find(b"tEXtRAM")
        end = b.find(b"tEXtBP1")
        if start == -1 or end == -1:
            return None
        raw = b[start + len(b"tEXtRAM") + 1 : end - 8]
        raw = raw.replace(b"\r\n", b"\r")
        if len(raw) < 2048:
            raw = raw + b"\x00" * (2048 - len(raw))
        return torch.tensor(list(raw[:2048]), dtype=torch.uint8)
    except Exception:
        return None

def ram_to_xt(ram: torch.Tensor):
    x = int(ram[0x6D]) * 256 + int(ram[0x86])
    timer = 100 * int(ram[0x07F8]) + 10 * int(ram[0x07F9]) + int(ram[0x07FA])
    return x, float(timer)

episodes = []
for d in sorted(p for p in data_root.rglob("*") if p.is_dir()):
    frames = sorted(d.glob("*.png"))
    if frames:
        episodes.append(frames)

if not episodes:
    raise RuntimeError(f"No episodes found under {data_root}")

samples = []
for frames in episodes:
    start = frameskip * (ctx - 1)
    for i in range(start, len(frames)):
        if len(samples) >= max_files:
            break
        seq = frames[i - frameskip * (ctx - 1) : i + 1 : frameskip]
        if len(seq) != ctx:
            continue
        rams = [read_ram(p) for p in seq]
        if any(r is None for r in rams):
            continue
        xs, ts = zip(*(ram_to_xt(r) for r in rams))
        dt = abs(ts[-1] - ts[0])
        if dt <= 0:
            dt = frameskip * (ctx - 1) / 60.0
        v = (xs[-1] - xs[0]) / max(dt, 1e-3)
        outcome = parse_outcome(seq[-1])
        samples.append({"seq": seq, "y": torch.tensor([v, ts[-1], outcome], dtype=torch.float32)})
    if len(samples) >= max_files:
        break
len(samples)



2000

In [ ]:
# quick scan for first usable window
probe = None
checked = 0
for frames in episodes:
    start = frameskip * (ctx - 1)
    for i in range(start, len(frames)):
        checked += 1
        seq = frames[i - frameskip * (ctx - 1) : i + 1 : frameskip]
        if len(seq) != ctx:
            continue
        rams = [read_ram(p) for p in seq]
        if any(r is None for r in rams):
            continue
        xs, ts = zip(*(ram_to_xt(r) for r in rams))
        dt = abs(ts[-1] - ts[0])
        if dt <= 0:
            dt = frameskip * (ctx - 1) / 60.0
        v = (xs[-1] - xs[0]) / max(dt, 1e-3)
        outcome = parse_outcome(seq[-1])
        imgs = torch.stack([transform(Image.open(p).convert("RGB")) for p in seq])
        probe = {
            "seq": [str(p) for p in seq],
            "xs": xs,
            "ts": ts,
            "dt": dt,
            "v": v,
            "outcome": outcome,
            "imgs_shape": imgs.shape,
        }
        break
    if probe:
        break
probe, checked



({'seq': ['/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/extracted/Rafael_hwfrb471_e0_8-1_fail/Rafael_hwfrb471_e0_8-1_f1000_a148_2019-04-14_18-39-06.fail.png',
   '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/extracted/Rafael_hwfrb471_e0_8-1_fail/Rafael_hwfrb471_e0_8-1_f1004_a20_2019-04-14_18-39-06.fail.png',
   '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/extracted/Rafael_hwfrb471_e0_8-1_fail/Rafael_hwfrb471_e0_8-1_f1008_a20_2019-04-14_18-39-06.fail.png',
   '/content/drive/MyDrive/Colab Notebooks/Computer Vision/Projects/MLP/extracted/Rafael_hwfrb471_e0_8-1_fail/Rafael_hwfrb471_e0_8-1_f1011_a20_2019-04-14_18-39-06.fail.png'],
  'xs': (1579, 1585, 1591, 1596),
  'ts': (254.0, 254.0, 253.0, 253.0),
  'dt': 1.0,
  'v': 17.0,
  'outcome': 0.0,
  'imgs_shape': torch.Size([4, 3, 240, 240])},
 1)

In [ ]:
repo = "Flaaaande/mario-sd"
ckpt = hf_hub_download(repo_id=repo, filename="diffusion_pytorch_model.safetensors")
base = "runwayml/stable-diffusion-v1-5"
vae = AutoencoderKL.from_pretrained(base, subfolder="vae").to(device)
vae.load_state_dict(load_file(ckpt), strict=False)
vae.eval()
transform = transforms.Compose([
    transforms.Resize((HEIGHT, WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

@torch.no_grad()
def encode_latent(path: Path):
    img = transform(Image.open(path).convert("RGB")).unsqueeze(0).to(device)
    lat = vae.encode(img).latent_dist.sample() * vae.config.scaling_factor
    return lat.squeeze(0)

features, targets = [], []
for s in tqdm(samples):
    lats = torch.stack([encode_latent(p) for p in s["seq"]])
    features.append(lats.flatten().cpu())
    targets.append(s["y"])
X = torch.stack(features)
Y = torch.stack(targets)
in_dim = X.shape[1]



In [ ]:
class TensorDS(Dataset):
    def __len__(self):
        return len(X)
    def __getitem__(self, i):
        return X[i], Y[i]

ds = TensorDS()
dl = DataLoader(ds, batch_size=64, shuffle=True)
model = nn.Sequential(
    nn.Linear(in_dim, 512), nn.ReLU(),
    nn.Linear(512, 256), nn.ReLU(),
    nn.Linear(256, 3),
).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(50):
    for xb, yb in dl:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = loss_fn(pred, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()
    print(epoch, loss.item())



0 5.695840835571289
1 8.757217407226562
2 0.477041095495224
3 0.7805594801902771
4 1.1514251232147217
5 0.8292170763015747
6 0.19456714391708374
7 0.5138733386993408
8 0.9853488206863403
9 1.005517601966858
10 0.273840993642807
11 0.4657316207885742
12 0.10762685537338257
13 0.11676832288503647
14 0.08852837979793549
15 0.09736816585063934
16 0.17085230350494385
17 0.11232416331768036
18 0.1040743738412857
19 0.11765646934509277
20 0.3462471663951874
21 0.06422309577465057
22 0.17698198556900024
23 0.248561292886734
24 0.22888587415218353
25 0.3271954655647278
26 0.08421467989683151
27 0.03504249453544617
28 0.059994228184223175
29 0.037987805902957916
30 0.023037156090140343
31 0.05875500291585922
32 0.07859687507152557
33 0.07547441124916077
34 0.02427852526307106
35 0.025383930653333664
36 0.019668538123369217
37 0.019292427226901054
38 0.04792410135269165
39 0.07051438838243484
40 0.04020896181464195
41 0.08041723817586899
42 0.03313861042261124
43 0.06628832221031189
44 0.07284267

In [ ]:
torch.save({"model": model.state_dict(), "in_dim": in_dim}, "smb_mlp.pt")
token = os.getenv("HF_TOKEN")
repo = os.getenv("HF_REPO_ID")
if token and repo:
    from huggingface_hub import HfApi
    HfApi(token=token).upload_file(
        path_or_fileobj="smb_mlp.pt",
        path_in_repo="smb_mlp.pt",
        repo_id=repo,
        repo_type="model",
    )
"done"

